In [76]:
# from futile import Utils
# from futile import YamlArgparse as YAP
# reload(YAP)
# reload(PP)
# reload(Utils)

<module 'futile.Utils' from '/Users/dawson/Documents/CEACollaboration/SourceCode/Base/BuildGCC/install/lib/python2.7/site-packages/futile/Utils.pyc'>

# BigDFT Fragmentation 

In this notebook we will go through the steps needed for computing the purity indicator and spillage values. First, we define the calculator we will use.

In [16]:
from BigDFT import Calculators as C

In [17]:
code = C.SystemCalculator(omp=2, mpi_run="mpirun")

Initialize a Calculator with OMP_NUM_THREADS=2 and command mpirun /Users/dawson/Documents/CEACollaboration/SourceCode/PurifySteps/Build/install/bin/bigdft


In [18]:
code.update_global_options(skip=True)

## Setup Calculation

First we define the system, which will be two water molecules.

In [19]:
positions ="""
- O: [-0.931477999671, 0.004547999723, -0.117331000086]
- H: [-1.355753693973, -0.032108553467, 0.725181911626]
- H: [-0.003078395366, 0.011003111969, 0.053703839371]
- O: [1.500000000000, -0.003706000000, 0.113495000000]
- H: [1.796831000000, 0.762122000000, -0.351887000000]
- H: [1.773536000000, -0.747744000000, -0.399151000000]
"""

In [20]:
from futile import YamlIO as Y
posinp = {"positions": Y.load(stream=positions, doc_lists=False),
          "units":"angstroem"}

We will next define an input file for doing a calculation. For reference, let's do a run using both the linear and the cubic scaling version.

In [21]:
from BigDFT import Inputfiles as I

In [22]:
inp = I.Inputfile()
inp.set_xc("PBE")
inp.set_hgrid(0.4)
inp.write_orbitals_on_disk()
inp.calculate_dipole()
inp["import"] = "linear"

In [24]:
full_qm = code.run(name="linear", input=inp, posinp=posinp, skip=False)

Creating the yaml input file "./linear.yaml"
Executing command:  mpirun /Users/dawson/Documents/CEACollaboration/SourceCode/PurifySteps/Build/install/bin/bigdft -n linear


## Fragmentation Study

In [26]:
from BigDFT import Fragments as F

In [27]:
dimer = F.System(mp_dict=full_qm.electrostatic_multipoles, nat_reference=3)

In [28]:
dimer.dict()
first, second = dimer.fragments
print(first.Q(), second.Q())
print(first.Q()+second.Q())

(-8.1159716, -7.8840268)
-15.9999984


In [29]:
print(first)

<BigDFT.Fragments.Fragment instance at 0x10a8431b8>


In [30]:
from BigDFT import PostProcessing as PP

In [31]:
btool = PP.BigDFTool()

In [33]:
btool.get_fragment_multipoles(full_qm)